In [1]:
import tensorflow as tf
old_v = tf.logging.get_verbosity()
tf.logging.set_verbosity(tf.logging.ERROR)

<h2>Extract MNIST data</h2>
<p style="font-size:20px">You can change the option of one_hot encoding.

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
#get mnist data, with one_hot encoding
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
#suppress warnings
tf.logging.set_verbosity(old_v)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


<h2>Define hyperparameters</h2>

In [28]:
num_train = mnist.train.num_examples #55,000
num_validation = mnist.validation.num_examples #5000
num_test = mnist.test.num_examples #10,000

#learning rate
lr = 0.01
#number of traning steps
num_steps = 1000
#number of batch_size
batch_size = 128

#network parameters
n_hidden_1 = 18000
n_hidden_2 = 6000
#n_hidden_3 = 1000
num_input = 784
num_classes = 10

<h2>Define placeholder and Variables</h2>

In [29]:
tf.reset_default_graph()


#tf graph input
X = tf.placeholder(tf.float32,[None,num_input],name='X')
Y = tf.placeholder(tf.int32,[None,num_classes],name='Y')

#Layers weight & bias
weights = {
    'W1': tf.Variable(tf.random_normal([num_input, n_hidden_1]),name='W1'),
    'W2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]),name='W2'),
    #'W3': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_3]),name='W3'),
    'Wout': tf.Variable(tf.random_normal([n_hidden_2, num_classes]),name='Wout')
}

biases = {
    'b1': tf.Variable(tf.zeros(shape=[n_hidden_1]),name='b1'),
    'b2': tf.Variable(tf.zeros(shape=[n_hidden_2]),name='b2'),
    #'b3': tf.Variable(tf.zeros(shape=[n_hidden_3]),name='b3'),
    'bout': tf.Variable(tf.zeros(shape=[num_classes]),name='bout')
}

<h2>Define neural network</h2>

In [30]:
#define a neural net model
def neural_net(x):
    layer_1_out = tf.nn.relu(tf.add(tf.matmul(x,weights['W1']),biases['b1']))
    layer_2_out = tf.nn.relu(tf.add(tf.matmul(layer_1_out,weights['W2']),biases['b2']))
    #layer_3_out = tf.nn.relu(tf.add(tf.matmul(layer_2_out,weights['W3']),biases['b3']))
    out = tf.add(tf.matmul(layer_2_out,weights['Wout']),biases['bout'])
    return out

<h2>Define cost function and accuracy</h2>

In [31]:
#predicted labels
logits = neural_net(X)

#define loss
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits,labels=Y),name='loss')
#define optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
train_op = optimizer.minimize(loss)

#compare the predicted labels with true labels
correct_pred = tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))

#compute the accuracy by taking average
accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32),name='accuracy')

<h2>Execute training</h2>

In [32]:
#Initialize the variables
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for i in range(num_steps):
        #fetch batch
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        #run optimization
        sess.run(train_op, feed_dict={X:batch_x, Y:batch_y})
        if i % (num_steps / 25) ==0:
            acc = sess.run(accuracy,feed_dict={X:batch_x, Y:batch_y})
            print("step "+str(i)+", Accuracy= {:.3f}".format(acc))
    
    print("Training finished!")
    
    print("Testing Accuracy:", sess.run(accuracy, feed_dict={X:mnist.test.images, Y:mnist.test.labels}))

step 0, Accuracy= 0.289
step 40, Accuracy= 0.539
step 80, Accuracy= 0.969
step 120, Accuracy= 0.984
step 160, Accuracy= 0.977
step 200, Accuracy= 1.000
step 240, Accuracy= 0.953
step 280, Accuracy= 0.969
step 320, Accuracy= 0.992
step 360, Accuracy= 0.992
step 400, Accuracy= 0.992
step 440, Accuracy= 0.992
step 480, Accuracy= 1.000
step 520, Accuracy= 0.984
step 560, Accuracy= 0.969
step 600, Accuracy= 1.000
step 640, Accuracy= 0.977
step 680, Accuracy= 0.984
step 720, Accuracy= 1.000
step 760, Accuracy= 0.992
step 800, Accuracy= 1.000
step 840, Accuracy= 0.992
step 880, Accuracy= 1.000
step 920, Accuracy= 1.000
step 960, Accuracy= 1.000
Training finished!
Testing Accuracy: 0.9623


Accuracy at different epochs and the accuracy of the testing set.

<h2>Your results</h2>

|Lr	    |Epoch|	Activation|	Batch size|	HL 1 nodes|	HL 2 nodes|	Accuracy|
|------|------|------|------|------|------|------|
|0.01	|500|	    None|	    128|	        300|	        100|	        0.84|
|0.01	|50000|	Sigmoid	|    128|	        300|	        100	  |      0.88|
|0.001	|50000|	Sigmoid	|    128|	        300	 |       100	 |       0.71|
|0.01	|50000|	ReLu	|    128|	        300	 |       100	 |       0.89|
|0.001	|500000|	ReLU|	    128|	300|	100|	0.904|
|0.01	|5000|	ReLU	|    128|	3000|	1000|	0.929|
|0.01	|10000|	ReLU	|    128|	3000|	1000|	0.934|
|0.001	|50000|	ReLU	|    128|	3000|	1000|	0.948|
|0.01	|10000|	ReLU	|    128|	6000|	2000|	0.9511|
|0.01	|1000|	ReLU	|    128|	9000|	3000|	0.9473|
|0.01	|1000|	ReLU	|    128|	12000|	4000|	0.9537|
|0.01	|1000|	ReLU	|    128|	15000|	5000|	0.9608|
|0.01	|1000|	ReLU	|    128|	18000|	6000|	0.9623|


In trying to get the accuracy of the testing data above 96% several attributes were modified and tested. These included the learning rate, the number of epochs, the activation functions, and the number of nodes in the hidden layers. The modifications that made the biggest difference were adding more nodes to the hidden layers and adding activation functions to the hidden layers. These changes improving the behavior of the system makes reasonable sense. Originally the system didn’t have activation functions between the layers, so the system was just summing everything together. Adding the activation functions completed the neural network's structure and let us get the full benefit of the two hidden layers. Adding more neurons improved the behavior because as more neurons were added the system gained more “thinking” power and the network could develop a better and more generalized classifier, allowing it to do better on the testing data. Decreasing the learning rate, increasing the number of epochs, and using ReLU over sigmoid only had minor to no benefit for performance. The minimal benefit of increasing the training time makes sense as the primary issue was that the system had high variance meaning the training accuracy was much better than the testing accuracy. In this scenario it is much more beneficial to make the network bigger than to increase the training time. 
